In [1]:
#!/bin/python3

#%%
import tensorflow as tf
import keras
import numpy as np
import matplotlib.pyplot as plt
import os
import keras_tuner as kt
import sklearn as skl
from sklearn.model_selection import train_test_split

print('Tensorflow version: ' + str(tf.__version__))
print('Keras version: ' + str(keras.__version__))
print('KerasTuner version: ' + str(kt.__version__))
print('Scikit-learn version: ' + str(skl.__version__))

Tensorflow version: 2.15.0
Keras version: 3.0.5
KerasTuner version: 1.4.6
Scikit-learn version: 1.3.0


In [2]:
# load the datasets
path = './datasets/GTSRB/'

print('Loading datasets...')
training = np.load(os.path.join(path, 'training.npz'), allow_pickle=True)
testing = np.load(os.path.join(path, 'testing.npz'), allow_pickle=True)

X_train = training['X']
y_train = training['y']

X_test = testing['X']
y_test = testing['y']

Loading datasets...


In [3]:
# converting datatypes
X_train = np.asarray(X_train).astype('float32')
X_test = np.asarray(X_test).astype('float32')

In [4]:
# split training data into training and validation set
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2,
                                                      shuffle=True)

In [5]:
# augment data

def augment_data(X):
    layers = [
        tf.keras.layers.RandomRotation(1/8), # 45 degrees, either cw/ccw
        tf.keras.layers.RandomTranslation(0.1, 0.1) # shift w/h by up to 10%
    ]

    X_augmented = []

    for x in X:
        for layer in layers:
            x = layer(x)
        
        X_augmented.append(x)
    
    return np.asarray(X_augmented).astype('float32')

print('Augmenting training and validation data...')
X_train = tf.py_function(func=augment_data, inp=[X_train], Tout=tf.float32)
X_valid = tf.py_function(func=augment_data, inp=[X_valid], Tout=tf.float32)

Augmenting training and validation data...


In [6]:
# build model
print('Searching and building a model...')

def build_model(hp):
    activation_functions = ['relu', 'sigmoid', 'softmax', 'softplus', 'softsign', 'tanh',
                            'selu', 'elu', 'exponential', 'leaky_relu', 'relu6', 'silu',
                            'gelu', 'hard_sigmoid', 'log_softmax', 'mish', 'linear']
    units_l0 = hp.Int('units_l0', 256, 1024, 128)
    activation_l0 = hp.Choice('activation_l0', activation_functions)
    units_l1 = hp.Int('units_l1', 256, 1024, 128)
    activation_l1 = hp.Choice('activation_l1', activation_functions)
    dropout_l2 = hp.Float('dropout_l2', 0.0, 0.5)
    activation_final = hp.Choice('activation_final', activation_functions)

    model = tf.keras.Sequential([
        tf.keras.layers.Input(shape=(50, 50, 3)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units_l0,
                              activation=activation_l0),
        tf.keras.layers.Dense(units_l1,
                              activation=activation_l1),
        tf.keras.layers.Dropout(dropout_l2),
        tf.keras.layers.Dense(43, activation=activation_final) # 43 classes
    ])

    loss_from_logits = True
    if activation_final in ['sigmoid', 'softmax']:
        loss_from_logits = False
    
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=loss_from_logits),
                  metrics=['accuracy'])
    return model

tuner = kt.BayesianOptimization(build_model,
                        project_name='kt_simple_model',
                        objective='val_accuracy',
                        max_trials=20,
                        seed=42
                        )
tuner.search(X_train, y_train, epochs=5, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)
])
tuner.results_summary()
model = tuner.get_best_models()[0]

model.summary()

Trial 20 Complete [00h 00m 20s]
val_accuracy: 0.42696627974510193

Best val_accuracy So Far: 0.42696627974510193
Total elapsed time: 00h 13m 24s
Results summary
Results in .\kt_simple_model
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 19 summary
Hyperparameters:
units_l0: 256
activation_l0: leaky_relu
units_l1: 640
activation_l1: relu6
dropout_l2: 0.3630456668613308
activation_final: linear
Score: 0.42696627974510193

Trial 09 summary
Hyperparameters:
units_l0: 512
activation_l0: softplus
units_l1: 512
activation_l1: selu
dropout_l2: 0.07283462755206516
activation_final: leaky_relu
Score: 0.3968699872493744

Trial 02 summary
Hyperparameters:
units_l0: 1024
activation_l0: sigmoid
units_l1: 1024
activation_l1: relu
dropout_l2: 0.29037858246697834
activation_final: softplus
Score: 0.19783306121826172

Trial 03 summary
Hyperparameters:
units_l0: 512
activation_l0: exponential
units_l1: 1024
activation_l1: elu
dropout_l2: 0.23807913111467283
activation_final

c:\Users\leona\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 14 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 7500)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 256)            │     1,920,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 640)            │       164,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 640)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 43)             │        27,563 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,112,299 (8.06 MB)

 Trainable params: 2,112,299 (8.06 MB)

 Non-trainable params: 0 (0.00 B)

In [7]:
# fit model
print('Fitting model...')
history = model.fit(X_train, y_train, epochs=50, validation_data=(X_valid, y_valid), callbacks=[
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5,
                                     restore_best_weights=True)
])

Fitting model...
Epoch 1/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 5s 12ms/step - accuracy: 0.4224 - loss: 1.8535 - val_accuracy: 0.4639 - val_loss: 1.6807
Epoch 2/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4476 - loss: 1.7470 - val_accuracy: 0.4872 - val_loss: 1.6263
Epoch 3/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4620 - loss: 1.6736 - val_accuracy: 0.4884 - val_loss: 1.6272
Epoch 4/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.4749 - loss: 1.6256 - val_accuracy: 0.5225 - val_loss: 1.5098
Epoch 5/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5160 - loss: 1.4978 - val_accuracy: 0.5265 - val_loss: 1.5000
Epoch 6/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5170 - loss: 1.4719 - val_accuracy: 0.5293 - val_loss: 1.4252
Epoch 7/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5405 - loss: 1.4172 - val_accuracy: 0.5325 - val_loss: 1.4399
Epoch 8/50
312/312 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.5517 - loss: 

In [8]:
# save model
model.save('simple_model.keras')

In [9]:
# load model
model = tf.keras.models.load_model('simple_model.keras')

In [10]:
# testing
test_loss, test_acc = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy:', test_acc)

121/121 - 0s - 3ms/step - accuracy: 0.6204 - loss: 1.3253
Test accuracy: 0.6204113364219666
Test accuracy: 0.6204113364219666
